##Pips

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bnaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.9/23.9 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transformers-4.26.1


In [ ]:
!wget https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
!wget https://huggingface.co/sagorsarker/bangla-glove-vectors/resolve/main/bn_glove.300d.zip

!unzip bangla_word2vec_gen4.zip
!unzip bn_glove.300d.zip

!rm -rf bangla_word2vec_gen4.zip
!rm -rf bn_glove.300d.zip

--2023-02-28 06:52:31--  https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f5/12/f51294acf565f8e7f2b231a61a327e6a79fce680615e3bbbc4850f166e928650/b157fe019ddda848f0195240f636ff2dcb63cbe823c84eed77c129e196105fd1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27bangla_word2vec_gen4.zip%3B+filename%3D%22bangla_word2vec_gen4.zip%22%3B&response-content-type=application%2Fzip&Expires=1677826352&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Y1LzEyL2Y1MTI5NGFjZjU2NWY4ZTdmMmIyMzFhNjFhMzI3ZTZhNzlmY2U2ODA2MTVlM2JiYmM0ODUwZjE2NmU5Mjg2NTAvYjE1N2ZlMDE5ZGRkYTg0OGYwMTk1MjQwZjYzNmZmMmRjYjYzY2JlODIzYzg0ZWVkNzdjMTI

## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from transformers import *


In [ ]:
from bnaug.sentence import BackTranslation

punkt not found. downloading...


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import logging
logging.disable(logging.INFO) # disable INFO and DEBUG logging everywhere

##function definitions

In [ ]:
def calc_len(sent):
  word_count = len(sent.split())
  return word_count

In [ ]:
def split_sent_daari(str_w):
  x = str_w.rsplit("। ")
  n = len(x)
  if n==1:
    x = str_w.rsplit("।")
    n = len(x)
  if n==1:
    x = str_w.rsplit(", ")
  if n==1:
    x = str_w.rsplit("? ")
  if n==1:
    return -1

  return x
    

In [ ]:
def removeOccurrences(self, s: str, part: str) -> str:       
  while part in s:     
    s = s[:s.index(part)] + s[s.index(part)+len(part):]     
  return s
  # a_sent = removeOccurrences(a_sent, a_sent, '<unk>')

In [ ]:
#,

##Loading data

Load your DS in this section. You may need to modify this section according to your DS attributes. Also note that there are two sub sections - initial iteration and next iterations. This is because we had to augment whole DS in batches as google collab runs for a limited time. 

Run the initial iteration only the first time. And do not run "next iteration" in first time.

In each subsequent times, do not run initial iteration, run only the next iteration. After the main code section, an output is generated of how much data has been augmented.

###initial iteration
don't run this sub-section after initial iteration

In [ ]:
path1 = '/content/drive/MyDrive/datasets/xlsum_sp3.csv'

In [ ]:
df = pd.read_csv(path1)

In [ ]:
split1 = df
df.head(2)

,id,summary
0,news-51894435,করোনাভাইরাস প্রতিরোধে ইউরোপীয় ইউনিয়নের বৃহৎ ...
1,news-45634902,ইন্দোনেশিয়ার এক তরুণ গভীর সাগরে ভেসে ছিলেন টা...


In [ ]:
split1.rename(columns = {'summary':'text'}, inplace = True)
split1.head(2)

,id,text
0,news-51894435,করোনাভাইরাস প্রতিরোধে ইউরোপীয় ইউনিয়নের বৃহৎ ...
1,news-45634902,ইন্দোনেশিয়ার এক তরুণ গভীর সাগরে ভেসে ছিলেন টা...


In [ ]:
a_sent = split1['text'][0]
a_sent

'করোনাভাইরাস প্রতিরোধে ইউরোপীয় ইউনিয়নের বৃহৎ দুই দেশ স্পেন ও ফ্রান্সও ইটালির পথ অনুসরণ করে জরুরি বিধিনিষেধ আরোপ করলো।'

###next iterations

In [ ]:
# path2 = '/content/drive/MyDrive/datasets/xlsum_sp3_bt.csv'
# split1 = pd.read_csv(path2)

##Data Augmentation

###loading augmenter and an example

In [ ]:
bt = BackTranslation()

In [ ]:
a_sent = 'মুরগির হামলায় শেয়াল নিহত'

In [ ]:
bt.get_augmented_sentences(a_sent)

[' মুরগির আক্রমণে শেয়াল নিহত']

In [ ]:
split_sents = split_sent_daari(a_sent)
split_sents

-1

In [ ]:
#individually backtranslate each sentences
for i in range(len(split_sents)):
  split_sents[i] = bt.get_augmented_sentences(split_sents[i])


In [ ]:
split_sents

[[' ইউরোপীয় ইউনিয়নের দুই বৃহত্তম দেশ স্পেন ও ফ্রান্সও ইতালির মতো করে জরুরি বিধিনিষেধ আরোপ করেছে।'],
 [' কিন্তু, তা সত্ত্বেও আমরা এই বিষয়ে নিশ্চিত থাকতে পারি, আমরা সেই সময়েও যিহোবার সেবা করে যাব।']]

In [ ]:
#original sentence
a_sent

'করোনাভাইরাস প্রতিরোধে ইউরোপীয় ইউনিয়নের বৃহৎ দুই দেশ স্পেন ও ফ্রান্সও ইটালির পথ অনুসরণ করে জরুরি বিধিনিষেধ আরোপ করলো।'

In [ ]:
#join all sentences to make one single string
result = ''.join(''.join(l) for l in split_sents) #nested join
result

' ইউরোপীয় ইউনিয়নের দুই বৃহত্তম দেশ স্পেন ও ফ্রান্সও ইতালির মতো করে জরুরি বিধিনিষেধ আরোপ করেছে। কিন্তু, তা সত্ত্বেও আমরা এই বিষয়ে নিশ্চিত থাকতে পারি, আমরা সেই সময়েও যিহোবার সেবা করে যাব।'

###batch augment whole DS

In [ ]:
split1.head()

,id,text
0,news-51894435,করোনাভাইরাস প্রতিরোধে ইউরোপীয় ইউনিয়নের বৃহৎ ...
1,news-45634902,ইন্দোনেশিয়ার এক তরুণ গভীর সাগরে ভেসে ছিলেন টা...
2,news-51003775,ইরান ঘোষণা করেছে যে তারা আর ২০১৫ সালের পরমাণু ...
3,news-50675121,ব্রিটিশ সাংস্কৃতিক সংগঠন 'সিটি লিট' বলেছে তাদে...
4,news-44090968,আলঝেইমার্স (মস্তিস্কের এক ধরনের রোগ যার ফলে কি...


In [ ]:
split1['text'][34]

'ইউরোপীয় ইউনিয়নের (ইইউ) একটি অভ্যন্তরীন ফ্লাইটকে রবিবার যাত্রাপথ পরিবর্তন করতে বাধ্য করার ঘটনায় রাগত প্রতিক্রিয়া ব্যক্ত করেছে পশ্চিমা দেশগুলো। গ্রিস থেকে লিথুয়ানিয়াগামী রায়ানএয়ারের এই ফ্লাইটটিতে বেলারুশের একজন ভিন্নমতের সাংবাদিক ছিলেন।'

In [ ]:
split1.shape

(500, 2)

**Main code section**

In [ ]:
#start denotes starting position in your DS and the ending position is set to final position
start = 0 #done till 975
for itr in range(start, len(split1)):
  sent = split1['text'][itr] 
  
  #splitting the sentence according to stop words
  split_sents = split_sent_daari(sent)
  if(split_sents==-1):
    print(itr, 'th row has error. Has not been split properly')
    continue
  
  for i in range(len(split_sents)):
    split_sents[i] = bt.get_augmented_sentences(split_sents[i])
  
  result = ''.join(''.join(l) for l in split_sents) #nested join

  split1.at[itr, 'text'] = result

  # print(split1['text'][itr])
  
  #on every 25th row, making a save
  if(itr%10==0):
    split1.to_csv('/content/drive/MyDrive/datasets/xlsum_sp3_bt.csv', index=False) #bt = backtranslation
    print(itr,' made a save')
  else:
    print(itr, end=' ')

0  made a save
10  made a save
20  made a save
30  made a save
31 th row has error. Has not been split properly
35 th row has error. Has not been split properly
40  made a save
50  made a save
60  made a save
62 th row has error. Has not been split properly
70  made a save
80  made a save
90  made a save
98 th row has error. Has not been split properly
100  made a save
110  made a save
120  made a save
124 th row has error. Has not been split properly
130  made a save
140  made a save
150  made a save
160  made a save
170  made a save
180  made a save
190  made a save
200  made a save
210  made a save
220  made a save
230  made a save
240  made a save
250  made a save
260  made a save
270  made a save
274 th row has error. Has not been split properly
280  made a save
290  made a save
300 th row has error. Has not been split properly
310  made a save
320  made a save
330  made a save
340  made a save
350  made a save
360  made a save
369 th row has error. Has not been split properly
370

Sometimes Daari or coma creates error. So you have to singularly check where there's error shown in the above output.

In [ ]:
itr = 492 #singular checks
sent = split1['text'][itr] 

#splitting the sentence according to stop words
split_sents = split_sent_daari(sent)

split_sents

['(সৌদি আরবের ঘটনাবলী নিয়ে বিবিসির নিরাপত্তা বিষয়ক সংবাদদাতা ফ্রাঙ্ক গার্ডনারের প্রতিবেদন)']

In [ ]:
split1.at[itr, 'text'] = bt.get_augmented_sentences(sent)
print(itr, split1.at[itr, 'text'])

492 [' (সৌদি আরবের ঘটনাবলী নিয়ে বিবিসির নিরাপত্তা বিষয়ক সংবাদদাতা ফ্রাঙ্ক গার্ডনারের প্রতিবেদন)']


In [ ]:
for i in range(len(split_sents)):
  split_sents[i] = bt.get_augmented_sentences(split_sents[i])
  
result = ''.join(''.join(l) for l in split_sents) #nested join

split1.at[itr, 'text'] = result
result

' আপনি কি আজ অন্তত কিছুটা পথ হেঁটেছেন অফিসে কাজ কি বসে বসে করতে হয়, নাকি হাতে হাতে করতে হয় আজ কোন খেলা করেছেন?'

In [ ]:
#done. save all.
split1.to_csv('/content/drive/MyDrive/datasets/xlsum_sp3_bt.csv', index=False) #bt = backtranslation
print('saved all')

saved all
